In [97]:
import sys
import numpy as np
from pathlib import Path
from itertools import combinations
# Define path to chart
d = str(Path().resolve().parent)
sys.path.insert(1, d + r'\Preprocessing')

from chart_functions import chart2dict, get_configuration

# Import the processed .chart file
chartpath = d+'\Preprocessing\Chart Files\degausser_notes.chart'
folderpath = d+'\Prototypes\Toy Training Data\Brand New - Degausser (Chezy)'

notes, song_metadata, time_signatures, BPMs = chart2dict(chartpath)
config = get_configuration(folderpath)

Still need to account for held notes

In [98]:
# Create a dictionary where the keys are the tick values and the values
    # are a list of notes corresponding to the data
coded_notes_0 = {}

# Loop through song one note at a time, processing along the way
for i in range(len(notes['tick'])):
    if notes['N_S'] == 'S':  # if the token is a star power indicator, skip it
        continue

    if notes['tick'][i] not in coded_notes_0:  # If the key is not in the dictionary
        coded_notes_0[notes['tick'][i]] = []                   # Create empty list

    if notes['duration'][i] == 0:  # If the note is not held
        coded_notes_0[notes['tick'][i]].append(int(notes['note'][i]))  # Add note to list
    else:  # If the note has some duration
        if (notes['tick'][i] + notes['duration'][i]) not in coded_notes_0:  # If the key is not in the dictionary
            coded_notes_0[notes['tick'][i] + notes['duration'][i]] = []     # Create empty list
        # Add the note with a hold key
        # Hold key is the code of the note + 10 (so 11 indicates release green, 12 is release red, etc.)
        coded_notes_0[notes['tick'][i]].append(int(notes['note'][i]) + 10)  # Add note to list

        # Add a release key at time step when note will be released.
        # Release key is the code of the note + 20 (so 21 indicates release green, 22 is release red, etc.)
        coded_notes_0[notes['tick'][i] + notes['duration'][i]].append(int(notes['note'][i]) + 20)  # Add note to list

In [99]:
print(coded_notes_0)

{605: [1], 681: [1], 792: [1], 903: [1], 1013: [1, 2], 1124: [1], 1234: [1], 1345: [1, 2], 1455: [1], 1566: [1], 1677: [1], 1787: [1], 1898: [1, 2], 2008: [1], 2119: [1, 2], 2230: [7], 2340: [0, 2], 2451: [0], 2561: [0], 2672: [0], 2783: [0], 2893: [0], 3004: [0], 3114: [0], 3225: [0], 3336: [0, 2], 3446: [0, 2], 3557: [0, 2], 3667: [0, 2], 3778: [0, 2], 3888: [0, 2], 3999: [7], 4110: [1], 4220: [1], 4331: [1], 4441: [1], 4552: [1, 2], 4663: [1], 4773: [1], 4884: [1, 2, 4], 4994: [1], 5105: [1], 5216: [1], 5326: [1], 5437: [1, 2], 5547: [1], 5658: [1, 2, 4], 5769: [7], 5879: [0, 2], 5990: [0], 6100: [0], 6211: [0], 6322: [0], 6432: [0], 6543: [0], 6653: [0, 2], 6764: [0], 6874: [0, 2], 6985: [0, 2], 7096: [0], 7206: [0, 2], 7317: [0, 2], 7427: [0, 2], 7538: [7], 7649: [11, 12], 7815: [21], 9418: [22, 10], 7870: [2], 7980: [0], 8091: [1], 8202: [4], 8312: [1], 8423: [1], 8533: [3], 8644: [1], 8755: [1], 8865: [12], 9363: [22], 9584: [20], 9639: [1], 9750: [7], 9860: [0], 9971: [2], 1008

## how to read coded_notes
coded_notes is a dictionary with the keys representing ticks and values decoded as follows:

| Value | Interpretation | Value | Interpretation |
| --- | --- | ---|---|
| 0 | green note | 5 | force note flag
| 10 | green note hold | 6 | tap note flag
| 20 | green note release |
| 1 | red note | 
| 11 | red note hold |
| 21 | red note release |
| 2 | yellow note |
| 12 | yellow note hold |
| 22 | yellow note release | 
| 3 | blue note | 
| 13 | blue note hold |
| 23 | blue note release |
| 4 | orange note |
| 14 | orange note hold |
| 24 | orange note release |
| 7 | open note

The flags are always listed after a note value. For example, a 10 followed by a 5 says 'hold a tapped green note.'

## next steps
One hot encode the 'bag of notes'. Translate coded_notes into a tensor with the features *timestamp, note event*, with a timestamp existing for every 10ms of the song.

We will translate as follows:

| Value | Interpretation | regular/force/tap |
| --- | --- | ---|
|0 | no note |
| 10 | green note | regular |
| 11 | green note | force |
| 12 | green note | tap |
| 13 | green note hold| regular |
| 14| green note hold| force |
| 15 | green note hold| tap |
| 16 | green note release |
| 17 | red note | regular |
| 18 | red note | force |
| 19 | red note | tap |
| 20 | red note hold | regular |
| 21 | red note hold | force |
| 22 | red note hold | tap |
| 23 | red note release |
| 24 | yellow note | regular |
| 25 | yellow note | force |
| 26 | yellow note | tap |
| 27 | yellow note hold | regular |
| 28 | yellow note hold | force |
| 29 | yellow note hold | tap |
| 30 | yellow note release | 
| 31 | blue note |regular |
| 32 | blue note | force|
| 33 | blue note | tap|
| 34 | blue note hold | regular |
| 35 | blue note hold |force
| 36 | blue note hold |tap
| 37 | blue note release |
| 38 | orange note |regular
| 39 | orange note |force
| 40 | orange note |tap
| 41 | orange note hold|regular
| 42 | orange note hold|force
| 43 | orange note hold|tap
| 44 | orange note release |
| 1 | open note | regular
| 2 | open note | force
| 3 | open note | tap

It is possible to have multiple notes per timestep (i.e. a chord), these will be dealt with in a subsequent step. First, this translation will be dubbed coded_notes_1

In [100]:
# Define function to map notes to their proper representation
def map_notes_0(note_array, note_type):
    '''
    map_notes_0 maps a note array from the initial representation to an intermediate
    representation that can be processed later into a full one hot representation.
    The note_array should be preprocessed so that force and tap flags are removed
    - note_array = array of notes
    - type = 'regular', 'force', or 'tap'
    '''
    assert note_type in ['regular', 'force', 'tap'], 'note_type should be "regular", "force", or "tap"'
    
    for i in range(len(note_array)):
        if note_array[i] == 0:
            note_array[i] = 10
        elif note_array[i] == 10:
            note_array[i] = 13
        elif note_array[i] == 20:
            note_array[i] = 16
        elif note_array[i] == 1:
            note_array[i] = 17            
        elif note_array[i] == 11:
            note_array[i] = 20
        elif note_array[i] == 21:
            note_array[i] = 23
        elif note_array[i] == 2:
            note_array[i] = 24
        elif note_array[i] == 12:
            note_array[i] = 27
        elif note_array[i] == 22:
            note_array[i] = 30
        elif note_array[i] == 3:
            note_array[i] = 31
        elif note_array[i] == 13:
            note_array[i] = 34
        elif note_array[i] == 23:
            note_array[i] = 37
        elif note_array[i] == 4:
            note_array[i] = 38
        elif note_array[i] == 14:
            note_array[i] = 41
        elif note_array[i] == 24:
            note_array[i] = 44
        elif note_array[i] == 7:
            note_array[i] = 1
        else:
            raise NameError('Error: note encoded incorrectly')

        if note_type == 'regular':
            continue
        elif note_type == 'force':
            note_array[i] += 1
        elif note_type == 'tap':
            note_array[i] += 2

    return note_array

In [101]:
# coded_notes_1 will hold intermediate values of coded_notes
coded_notes_1 = {}

for x in coded_notes_0.keys():
    if 5 in coded_notes_0[x]:    # If a force note
        coded_notes_0[x].remove(5)
        coded_notes_1[x] = map_notes_0(coded_notes_0[x], 'force')
    elif 6 in coded_notes_0[x]:  # If a tap note
        coded_notes_0[x].remove(6)
        coded_notes_1[x] = map_notes_0(coded_notes_0[x], 'tap')
    else:                        # If a regular note
        coded_notes_1[x] = map_notes_0(coded_notes_0[x], 'regular')


##FIXME

There is an error being thrown for the [30, 13] which means release yellow and hold green note at the same time
Need to find a workaround for this

In [105]:
i = 0
for k, v in coded_notes_1.items():
    print(k, ':', v)
    i+=1
    if i == 70:
        break

605 : [17]
681 : [17]
792 : [17]
903 : [17]
1013 : [17, 24]
1124 : [17]
1234 : [17]
1345 : [17, 24]
1455 : [17]
1566 : [17]
1677 : [17]
1787 : [17]
1898 : [17, 24]
2008 : [17]
2119 : [17, 24]
2230 : [1]
2340 : [10, 24]
2451 : [10]
2561 : [10]
2672 : [10]
2783 : [10]
2893 : [10]
3004 : [10]
3114 : [10]
3225 : [10]
3336 : [10, 24]
3446 : [10, 24]
3557 : [10, 24]
3667 : [10, 24]
3778 : [10, 24]
3888 : [10, 24]
3999 : [1]
4110 : [17]
4220 : [17]
4331 : [17]
4441 : [17]
4552 : [17, 24]
4663 : [17]
4773 : [17]
4884 : [17, 24, 38]
4994 : [17]
5105 : [17]
5216 : [17]
5326 : [17]
5437 : [17, 24]
5547 : [17]
5658 : [17, 24, 38]
5769 : [1]
5879 : [10, 24]
5990 : [10]
6100 : [10]
6211 : [10]
6322 : [10]
6432 : [10]
6543 : [10]
6653 : [10, 24]
6764 : [10]
6874 : [10, 24]
6985 : [10, 24]
7096 : [10]
7206 : [10, 24]
7317 : [10, 24]
7427 : [10, 24]
7538 : [1]
7649 : [20, 27]
7815 : [23]
9418 : [30, 13]
7870 : [24]
7980 : [10]
8091 : [17]


In [76]:
# To one hot encode, we need to generate a list of all the possible unique values each note event could take on
g = list(range(10,17))
r = list(range(17,24))
y = list(range(24,31))
b = list(range(31,38))
o = list(range(38,45))

note_vals = [g, r, y, b, o]
note_vals = np.array(note_vals)

rr = note_vals[:,0]  # regular regular
rf = note_vals[:,1]  # regular forced
rt = note_vals[:,2]  # regular tapped
hr = note_vals[:,3]  # held regular
hf = note_vals[:,4]  # held forced
ht = note_vals[:,5]  # held tapped
release = note_vals[:,6]  # release
note_combos = [rr, rf, rt, hr, hf, ht, release]

all_combinations = []  # Will hold all possible note combinations
for combo_class in note_combos:
    for combo_length in range(1, len(combo_class)+1):
        for combo in list(combinations(combo_class, combo_length)):
            keystring = ''
            for element in combo:
                keystring += str(element)
            all_combinations.append(keystring)

# Add open notes
all_combinations.append('1')
all_combinations.append('2')
all_combinations.append('3')

In [92]:
# Sanity check, are all the values in all_combinations unique?
unique_vals = len(list(set(all_combinations)))
print(len(all_combinations) == unique_vals)
print(all_combinations)

True
['10', '17', '24', '31', '38', '1017', '1024', '1031', '1038', '1724', '1731', '1738', '2431', '2438', '3138', '101724', '101731', '101738', '102431', '102438', '103138', '172431', '172438', '173138', '243138', '10172431', '10172438', '10173138', '10243138', '17243138', '1017243138', '11', '18', '25', '32', '39', '1118', '1125', '1132', '1139', '1825', '1832', '1839', '2532', '2539', '3239', '111825', '111832', '111839', '112532', '112539', '113239', '182532', '182539', '183239', '253239', '11182532', '11182539', '11183239', '11253239', '18253239', '1118253239', '12', '19', '26', '33', '40', '1219', '1226', '1233', '1240', '1926', '1933', '1940', '2633', '2640', '3340', '121926', '121933', '121940', '122633', '122640', '123340', '192633', '192640', '193340', '263340', '12192633', '12192640', '12193340', '12263340', '19263340', '1219263340', '13', '20', '27', '34', '41', '1320', '1327', '1334', '1341', '2027', '2034', '2041', '2734', '2741', '3441', '132027', '132034', '132041', '1

In [79]:
# coded_notes_2 will map the coded_notes_1 values into the syntax of the values described by all_combinations

coded_notes_2 = {}

for x in coded_notes_1.keys():
    notestring = ''
    for note_event in coded_notes_1[x]:
        notestring += str(note_event)
    coded_notes_2[x] = notestring

In [85]:
# combo_dictionary will map the coded values to a simpler representation [1:len(all_combinations)]

combo_dictionary = {}
for i in range(1, len(all_combinations)+1):
    combo_dictionary[all_combinations[i-1]] = i

In [93]:
# coded_notes_3 translates the coded notes into the simpler representations given by combo_dictionary
# combo_dictionary codes the combinations from 1-220

coded_notes_3 = {}
i = 0
for x in coded_notes_2.keys():
    print(i)
    coded_notes_3[x] = combo_dictionary[coded_notes_2[x]]
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


KeyError: '3013'

In [89]:
combo_dictionary['3013']

KeyError: '3013'